In [90]:
import pandas as pd
import datetime
import numpy as np
import time
import seaborn as sns
from lib import *

In [91]:
data = {
    'utc_timestamp': [],
    'direction': [],
    'rotation_duration_sec': []
}


In [92]:
time_step = datetime.timedelta(minutes = 0, seconds = 5)
now = datetime.datetime.now(datetime.timezone.utc)
now
obs_duration = datetime.timedelta(hours = 0, minutes = 10, seconds = 10)
end_time = now + obs_duration

In [93]:
times = np.random.choice(np.arange(now, end_time,  datetime.timedelta(minutes = 0, seconds = 0.5)), 100, replace=False)
data['utc_timestamp'] = np.sort(times)
data['direction'] = np.random.choice(['left', 'right'], len(times))
data['rotation_duration_sec'] = np.random.choice(np.arange(0, 8), len(times))

/var/folders/5q/9jqcz6pj3053zxqf_d22qb680000gr/T/ipykernel_3967/1414137269.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  times = np.random.choice(np.arange(now, end_time,  datetime.timedelta(minutes = 0, seconds = 0.5)), 100, replace=False)


In [94]:
obs_df = pd.DataFrame.from_dict(data)
obs_df

,utc_timestamp,direction,rotation_duration_sec
0,2024-10-17 06:39:25.572392,right,5
1,2024-10-17 06:39:26.072392,right,6
2,2024-10-17 06:39:27.072392,left,2
3,2024-10-17 06:39:27.572392,right,2
4,2024-10-17 06:39:32.572392,right,4
...,...,...,...
95,2024-10-17 06:49:02.572392,right,4
96,2024-10-17 06:49:16.072392,left,4
97,2024-10-17 06:49:21.572392,right,1
98,2024-10-17 06:49:27.572392,right,0


In [95]:
config = load_config()
obs_plan_dir = Path(config['obs_plan_dir'])
obs_plan_file = config['obs_plan_file']
obs_plan_path = obs_plan_dir / obs_plan_file
obs_plan_path

PosixPath('obs_plans/obs_plan_TEST')

In [96]:
obs_df.to_csv(obs_plan_path)
obs_df

,utc_timestamp,direction,rotation_duration_sec
0,2024-10-17 06:39:25.572392,right,5
1,2024-10-17 06:39:26.072392,right,6
2,2024-10-17 06:39:27.072392,left,2
3,2024-10-17 06:39:27.572392,right,2
4,2024-10-17 06:39:32.572392,right,4
...,...,...,...
95,2024-10-17 06:49:02.572392,right,4
96,2024-10-17 06:49:16.072392,left,4
97,2024-10-17 06:49:21.572392,right,1
98,2024-10-17 06:49:27.572392,right,0


In [97]:
obs_plan_loaded_df = pd.read_csv(obs_plan_path, index_col=0)
obs_plan_loaded_df['utc_timestamp'] = pd.to_datetime(obs_plan_loaded_df['utc_timestamp'], utc=True)
obs_plan_loaded_df

,utc_timestamp,direction,rotation_duration_sec
0,2024-10-17 06:39:25.572392+00:00,right,5
1,2024-10-17 06:39:26.072392+00:00,right,6
2,2024-10-17 06:39:27.072392+00:00,left,2
3,2024-10-17 06:39:27.572392+00:00,right,2
4,2024-10-17 06:39:32.572392+00:00,right,4
...,...,...,...
95,2024-10-17 06:49:02.572392+00:00,right,4
96,2024-10-17 06:49:16.072392+00:00,left,4
97,2024-10-17 06:49:21.572392+00:00,right,1
98,2024-10-17 06:49:27.572392+00:00,right,0


In [ ]:
now = datetime.datetime.now(datetime.timezone.utc)
obs_plan_loaded_df = obs_plan_loaded_df.sort_values(by='utc_timestamp')
scheduled_after_now = obs_plan_loaded_df[obs_plan_loaded_df['utc_timestamp'] >= now]
scheduling_errors = []

try:
    if len(scheduled_after_now) > 0:
        for idx in scheduled_after_now.index:
            next_action = scheduled_after_now.loc[idx]
            next_direction = next_action['direction']
            next_rotation_duration = next_action['rotation_duration_sec']
            scheduled_time = next_action['utc_timestamp']
            now = datetime.datetime.now(datetime.timezone.utc)
            seconds_until_scheduled_time = (scheduled_time - now).total_seconds()
            if seconds_until_scheduled_time < 0:  # in case of an unexpectedly long delay and action deadline has passed
                print(f'WARNING: MOVE DEADLINE PASSED BY {seconds_until_scheduled_time}')
                continue
            print(f'Movement {idx + 1:>7} of {len(obs_plan_loaded_df)}:') 
            print(f'\tSleeping {seconds_until_scheduled_time:>12.5}s until \t{scheduled_time}')
            time.sleep(seconds_until_scheduled_time)
            now = datetime.datetime.now(datetime.timezone.utc)
            scheduling_error = (scheduled_time - now).total_seconds()
            print(scheduling_error)
            scheduling_errors.append(scheduling_error)
            # print(f'DOING STUFF at {now}\n')
            print(f"\tRotating {next_direction.upper():>7} for {next_rotation_duration:>3}s @\t{now}\n")
            
            
        print('All movements completed')
    else:
        print('No scheduled actions after the current time')
finally:
    print('Exiting')


Movement       1 of 100:
	Sleeping       2.2524s until 	2024-10-17 06:39:25.572392+00:00
-0.005310000000000037
	Rotating   RIGHT for   5s @	2024-10-17 06:39:25.577702+00:00

Movement       2 of 100:
	Sleeping       0.4937s until 	2024-10-17 06:39:26.072392+00:00
-0.004628999999999994
	Rotating   RIGHT for   6s @	2024-10-17 06:39:26.077021+00:00

Movement       3 of 100:
	Sleeping      0.99404s until 	2024-10-17 06:39:27.072392+00:00
-0.004166000000000003
	Rotating    LEFT for   2s @	2024-10-17 06:39:27.076558+00:00

Movement       4 of 100:
	Sleeping      0.49513s until 	2024-10-17 06:39:27.572392+00:00
-0.0016220000000000123
	Rotating   RIGHT for   2s @	2024-10-17 06:39:27.574014+00:00

Movement       5 of 100:
	Sleeping       4.9947s until 	2024-10-17 06:39:32.572392+00:00
-0.004960999999999993
	Rotating   RIGHT for   4s @	2024-10-17 06:39:32.577353+00:00

Movement       6 of 100:
	Sleeping       3.9926s until 	2024-10-17 06:39:36.572392+00:00
-0.004566999999999988
	Rotating   RIGHT 

In [ ]:
sns.histplot(np.abs(scheduling_errors), stat='density')